In [69]:
import statistics
from scipy.stats import ttest_ind, ttest_rel
import numpy as np
from numpy import random
import pandas as pd
import io
import requests
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
import os
from pandas.api.types import CategoricalDtype
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.datasets import fetch_covtype
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification

In [4]:
covtype = pd.read_csv('Downloads\covtype.csv', sep=' *, *', na_values='?', engine='python')

In [5]:
index7 = covtype['Cover_Type'] == 7

In [6]:
index7.value_counts()

False    560502
True      20510
Name: Cover_Type, dtype: int64

In [7]:
Y_cov = np.where(index7, 1, 0)

In [8]:
pd.Series(Y_cov).value_counts(normalize=True)

0    0.9647
1    0.0353
dtype: float64

In [9]:
X_cov = covtype.drop(['Cover_Type'], axis=1)

In [10]:
Y_cov.shape

(581012,)

In [11]:
X_cov.shape

(581012, 54)

In [12]:
letter = pd.read_csv('Downloads\letter-recognition.csv', sep=' *, *', na_values='?', engine='python')

In [1]:
letter

NameError: name 'letter' is not defined

In [11]:
len(letter)

20000

In [12]:
letterO_Y = np.zeros(len(letter))
for i in range(len(letter)):
    if letter['letter'][i] == 'O':
        letterO_Y[i] += 1
    else:
        letterO_Y[i] += 0

In [13]:
pd.Series(letterO_Y).value_counts(normalize=True)

0.0    0.96235
1.0    0.03765
dtype: float64

In [14]:
Y_letterO = letterO_Y

In [19]:
X_letterO = letter.drop(['letter'], axis=1)
fit_lo = StandardScaler().fit(X_letterO)
X_letterO = fit_lo.transform(X_letterO)

In [16]:
AM = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
letterO_AM = np.zeros(len(letter))
for i in range(len(letter)):
    if letter['letter'][i] in AM:
        letterO_AM[i] +=1
    else:
        letterO_AM += 0

In [17]:
pd.Series(letterO_AM).value_counts(normalize=True)

0.0    0.503
1.0    0.497
dtype: float64

In [18]:
Y_letterAM = letterO_AM
X_letterAM = letter.drop(['letter'], axis=1)

In [19]:
## ANN
def ANN_classifier(X, Y):
    
    mean_test_acc = []
    
    opt_train_acc = []
    opt_train_f1= []
    opt_train_auc = []
    
    opt_test_acc = []
    opt_test_f1= []
    opt_test_auc = []

    for trial in range(5):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=5000,test_size=len(X)-5000,
                                                            stratify= Y, random_state=trial, shuffle=True)
        
        print(pd.Series(Y_train).value_counts())
        
        ann_param = [{'solver': ['sgd'], 'hidden_layer_sizes': [1,2,4,8,32,128], 'momentum': [0,0.2,0.5,0.9],
                      'class_weight':[{0: w} for w in [1, 2, 4, 6, 10]]}]
        scoring = ['accuracy', 'roc_auc', 'f1']
        
        ann_classifier = GridSearchCV(MLPClassifier(early_stopping =True), ann_param, cv=5, scoring = scoring,
                                      refit = False, n_jobs=1)
        ann_classifier.fit(X_train, Y_train)
        
        ## store best parameters for each metric
        best = []
        best_index = np.argmin(ann_classifier.cv_results_['rank_test_accuracy'])
        best_param = ann_classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(ann_classifier.cv_results_['rank_test_roc_auc'])
        best_param = ann_classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(ann_classifier.cv_results_['rank_test_f1'])
        best_param = ann_classifier.cv_results_['params'][best_index]
        best.append(best_param)
        
        mean_test = ann_classifier.cv_results_['mean_test_accuracy']
        mean_test_acc.append(mean_test)
        
        ## train samples using each best metric
        params_ann1=best[0]
        model1 = MLPClassifier(early_stopping=True).set_params(**params_ann1)
        model1.fit(X_train, Y_train)
    
        params_ann2=best[1]
        model2 = MLPClassifier(early_stopping=True).set_params(**params_ann2)
        model2.fit(X_train, Y_train)
    
        params_ann3=best[2]
        model3 = MLPClassifier(early_stopping=True).set_params(**params_ann3)
        model3.fit(X_train, Y_train)
        
        # get score for train set
        acc_train = []
        for i in model1, model2, model3:
            acc_train.append(accuracy_score(Y_train, i.predict(X_train)))
        opt_train_acc.extend(acc_train)
    
        f1_train = []
        for i in model1, model2, model3:
            f1_train.append(f1_score(Y_train, i.predict(X_train)))
        opt_train_f1.extend(f1_train)
        print(pd.Series(i.predict(X_train)).value_counts())
        
        auc_train = []
        for i in model1, model2, model3:
            auc_train.append(roc_auc_score(Y_train,i.predict(X_train)))
        opt_train_auc.extend(auc_train)
        
        ## get score for test set
        acc_test = []
        for i in model1, model2, model3:
            acc_test.append(accuracy_score(Y_test, i.predict(X_test)))
        opt_test_acc.extend(acc_test)
    
        f1_test = []
        for i in model1, model2, model3:
            f1_test.append(f1_score(Y_test,i.predict(X_test)))
        opt_test_f1.extend(f1_test)

        auc_test = []
        for i in model1, model2, model3:
            auc_test.append(roc_auc_score(Y_test,i.predict(X_test)))
        opt_test_auc.extend(auc_test)
    
    return best, mean_test_acc, opt_train_acc, opt_train_f1, opt_train_auc, opt_test_acc, opt_test_f1, opt_test_auc

In [23]:
def logistic_classifier(X, Y):
    
    mean_test_acc = []
    
    opt_train_acc = []
    opt_train_f1= []
    opt_train_auc = []
    
    opt_test_acc = []
    opt_test_f1= []
    opt_test_auc = []
    
    for trial in range(5):
    
        # define parameters
        C = [10**-8, 10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**1, 10**2, 10**3, 10**4]
        grid ={"C":C, "penalty":["l1","l2"]}
        scoring = ['accuracy', 'roc_auc', 'f1']
                    
        # pick random samples
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=len(X)-5000, train_size=5000, random_state=trial, shuffle=True)

        # define classifier, fit train Run Gridsearch
        classifier = GridSearchCV(LogisticRegression(solver = 'liblinear', class_weight = 'balanced'), grid, cv=5, scoring = scoring, refit = False, n_jobs=-1)
        classifier.fit(X_train, Y_train)
        
        # store best parameters for each metric for training and get accuracy score
        best = []
        best_index = np.argmin(classifier.cv_results_['rank_test_accuracy'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(classifier.cv_results_['rank_test_roc_auc'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(classifier.cv_results_['rank_test_f1'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        mean_test = classifier.cv_results_['mean_test_accuracy']
        mean_test_acc.append(mean_test)
    
        # train samples using each best metric
        model1 = LogisticRegression(C=best[0]['C'], penalty=best[0]['penalty'], solver='liblinear', class_weight='balanced')
        model1.fit(X_train, Y_train)
    
        model2 = LogisticRegression(C=best[1]['C'], penalty=best[1]['penalty'], solver='liblinear', class_weight = 'balanced')
        model2.fit(X_train, Y_train)
    
        model3 = LogisticRegression(C=best[2]['C'], penalty=best[2]['penalty'], solver='liblinear', class_weight = 'balanced')
        model3.fit(X_train, Y_train)
        
        # get score for train set
        acc_train = []
        for i in model1, model2, model3:
            acc_train.append(accuracy_score(Y_train, i.predict(X_train)))
        opt_train_acc.extend(acc_train)

        auc_train = []
        for i in model1, model2, model3:
            auc_train.append(roc_auc_score(Y_train, i.predict(X_train)))
        opt_train_auc.extend(auc_train)
        
        f1_train = []
        for i in model1, model2, model3:
            f1_train.append(f1_score(Y_train, i.predict(X_train)))
        opt_train_f1.extend(f1_train)
        
        # get score for test set
        acc_test = []
        for i in model1, model2, model3:
            acc_test.append(accuracy_score(Y_test, i.predict(X_test)))
        opt_test_acc.extend(acc_test)
    
        f1_test = []
        for i in model1, model2, model3:
            f1_test.append(f1_score(Y_test, i.predict(X_test))) 
        opt_test_f1.extend(f1_test)

        auc_test = []
        for i in model1, model2, model3:
            auc_test.append(roc_auc_score(Y_test, i.predict(X_test)))
        opt_test_auc.extend(auc_test)
    
    return best, mean_test_acc, opt_train_acc, opt_train_f1, opt_train_auc, opt_test_acc, opt_test_f1, opt_test_auc

In [24]:
logistic_classifier(X_letterO, Y_letterO)

([{'C': 1e-08, 'penalty': 'l1'},
  {'C': 0.01, 'penalty': 'l2'},
  {'C': 0.1, 'penalty': 'l1'}],
 [array([0.9594, 0.5096, 0.9594, 0.5098, 0.9594, 0.5104, 0.9594, 0.513 ,
         0.9594, 0.529 , 0.9594, 0.6466, 0.7272, 0.7564, 0.7794, 0.7772,
         0.7788, 0.779 , 0.7788, 0.7788, 0.7788, 0.7788, 0.7788, 0.7788]),
  array([0.9656, 0.503 , 0.9656, 0.503 , 0.9656, 0.5032, 0.9656, 0.5046,
         0.9656, 0.5208, 0.9656, 0.6374, 0.734 , 0.7438, 0.774 , 0.7726,
         0.7754, 0.775 , 0.775 , 0.775 , 0.775 , 0.775 , 0.775 , 0.775 ]),
  array([0.962 , 0.5236, 0.962 , 0.5236, 0.962 , 0.5236, 0.962 , 0.5254,
         0.962 , 0.5488, 0.8852, 0.6792, 0.7424, 0.7754, 0.7972, 0.7952,
         0.7972, 0.7972, 0.7972, 0.7972, 0.7972, 0.7972, 0.7972, 0.7972]),
  array([0.9608, 0.5406, 0.9608, 0.5406, 0.9608, 0.5406, 0.9608, 0.5428,
         0.9608, 0.5594, 0.8814, 0.6748, 0.7438, 0.7672, 0.7874, 0.7874,
         0.79  , 0.79  , 0.7902, 0.7902, 0.7902, 0.7902, 0.7902, 0.7902]),
  array([0.9636, 0.

In [ ]:
pd.Series(Y_letterO).value_counts()

In [ ]:
#weight of class1
len(Y_letterO)/(2*753)

In [ ]:
#of class 0
len(Y_letterO)/(2*19247)

In [ ]:
pd.Series(Y_train).value_counts()

In [ ]:
if X is X_letterO:
            weight = ['balanced']
            grid = {"C":C, "penalty":["l1","l2"], 'class_weight':weight}
        else:

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
def RF_classifier(X, Y):
    
    mean_test_acc = []
    
    opt_train_acc = []
    opt_train_f1= []
    opt_train_auc = []
    
    opt_test_acc = []
    opt_test_f1= []
    opt_test_auc = []
    
    for trial in range(5):
    
        # define parameters
        trees= [1024]
        max_features =[1,2,4,6,8,12,16, 20]
        grid = {'n_estimators': trees, 'max_features': max_features}
        scoring = ['accuracy', 'roc_auc', 'f1']
                    
        # pick random samples
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=len(X)-5000, train_size=5000, random_state=trial, shuffle=True)

        # define classifier, fit train Run Gridsearch
        classifier = GridSearchCV(RandomForestClassifier(class_weight = 'balanced'), grid, cv=5, scoring = scoring, refit = False, n_jobs=-1)
        classifier.fit(X_train, Y_train)
        
        # store best parameters for each metric for training and get accuracy score
        best = []
        best_index = np.argmin(classifier.cv_results_['rank_test_accuracy'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(classifier.cv_results_['rank_test_roc_auc'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        best_index = np.argmin(classifier.cv_results_['rank_test_f1'])
        best_param = classifier.cv_results_['params'][best_index]
        best.append(best_param)
    
        mean_test = classifier.cv_results_['mean_test_accuracy']
        mean_test_acc.append(mean_test)
    
        # train samples using each best metric
        model1 = RandomForestClassifier(max_features =best[0]['max_features'], class_weight='balanced')
        model1.fit(X_train, Y_train)
    
        model2 = RandomForestClassifier(max_features =best[1]['max_features'], class_weight = 'balanced')
        model2.fit(X_train, Y_train)
    
        model3 = RandomForestClassifier(max_features =best[2]['max_features'], class_weight = 'balanced')
        model3.fit(X_train, Y_train)
        
        # get score for train set
        acc_train = []
        for i in model1, model2, model3:
            acc_train.append(accuracy_score(Y_train, i.predict(X_train)))
        opt_train_acc.extend(acc_train)

        auc_train = []
        for i in model1, model2, model3:
            auc_train.append(roc_auc_score(Y_train, i.predict(X_train)))
        opt_train_auc.extend(auc_train)
        
        f1_train = []
        for i in model1, model2, model3:
            f1_train.append(f1_score(Y_train, i.predict(X_train)))
        opt_train_f1.extend(f1_train)
        
        # get score for test set
        acc_test = []
        for i in model1, model2, model3:
            acc_test.append(accuracy_score(Y_test, i.predict(X_test)))
        opt_test_acc.extend(acc_test)
    
        f1_test = []
        for i in model1, model2, model3:
            f1_test.append(f1_score(Y_test, i.predict(X_test))) 
        opt_test_f1.extend(f1_test)

        auc_test = []
        for i in model1, model2, model3:
            auc_test.append(roc_auc_score(Y_test, i.predict(X_test)))
        opt_test_auc.extend(auc_test)
    
    return best, mean_test_acc, opt_train_acc, opt_train_f1, opt_train_auc, opt_test_acc, opt_test_f1, opt_test_auc

In [34]:
RF_classifier(X_letterO, Y_letterO)

C:\Users\Jessica\Anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.9834 0.9866 0.9874 0.9874 0.9868 0.986  0.9864    nan]
  category=UserWarning
C:\Users\Jessica\Anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.996094   0.99671888 0.99618598 0.99529871 0.9947106  0.99197191
 0.99152652        nan]
  category=UserWarning
C:\Users\Jessica\Anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.74382833 0.80501676 0.82353147 0.82369396 0.81537334 0.80695308
 0.81359852        nan]
  category=UserWarning
C:\Users\Jessica\Anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.9858 0.9874 0.9884 0.9886 0.9878 0.9878 0.9872    nan]
 

([{'max_features': 4, 'n_estimators': 1024},
  {'max_features': 2, 'n_estimators': 1024},
  {'max_features': 6, 'n_estimators': 1024}],
 [array([0.9834, 0.9866, 0.9874, 0.9874, 0.9868, 0.986 , 0.9864,    nan]),
  array([0.9858, 0.9874, 0.9884, 0.9886, 0.9878, 0.9878, 0.9872,    nan]),
  array([0.9844, 0.9876, 0.9886, 0.9894, 0.989 , 0.989 , 0.9884,    nan]),
  array([0.9844, 0.988 , 0.9886, 0.9886, 0.9882, 0.9876, 0.9876,    nan]),
  array([0.9822, 0.9854, 0.9868, 0.9866, 0.986 , 0.986 , 0.986 ,    nan])],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.9899333333333333,
  0.9890666666666666,
  0.9904666666666667,
  0.9874666666666667,
  0.9870666666666666,
  0.9872666666666666,
  0.9891333333333333,
  0.9887333333333334,
  0.9892,
  0.989,
  0.9886,
  0.9890666666666666,
  0.989,
  0.98753333333333

In [35]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 0.24.1.


In [22]:
n_features = X_letterO.shape[1]
feature_size = [1,2,4,6,8,12,16, 20]
for i in feature_size:
    max_features = max(n_features, int(feature_size))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [31]:
##Table 2 mean over metrics
logi_acc = [0.8026204564666103,
  0.8000845308537616,
  0.8000845308537616,
  0.8045099696683408,
  0.8045099696683408,
  0.8045099696683408,
  0.8021480781661777,
  0.8006563572174432,
  0.8006563572174432,
  0.8009298393913779,
  0.7978220874148476,
  0.7978220874148476,
  0.805355278205957,
  0.805355278205957,
  0.805355278205957, ## adult
        0.9646847635118713,
  0.9331854197481997,
  0.932720151663507,
  0.9647125407109574,
  0.9225502246480977,
  0.9340222078706694,
  0.9647368457601577,
  0.919336749928821,
  0.9215137879071964,
  0.9646569863127852,
  0.9227880669152726,
  0.9310431032686819,
  0.9646864995868142,
  0.9273209585911405,
  0.9415046908744956, ## covertype
           0.9633333333333334,
  0.7642,
  0.7846,
  0.9612666666666667,
  0.7503333333333333,
  0.7694666666666666,
  0.9624666666666667,
  0.7783333333333333,
  0.7966666666666666,
  0.9628666666666666,
  0.7642,
  0.7836,
  0.9619333333333333,
  0.7596,
  0.7781333333333333, ##LetterO
           0.7244666666666667,
  0.7244666666666667,
  0.7244666666666667,
  0.7244666666666667,
  0.7268666666666667,
  0.7244666666666667,
  0.7257333333333333,
  0.7258666666666667,
  0.7256666666666667,
  0.7274666666666667,
  0.7275333333333334,
  0.7274666666666667,
  0.7328666666666667,
  0.7326,
  0.7328666666666667] ##letterAM
logi_acc_mean = statistics.mean(logi_acc)
print(logi_acc_mean)

0.826502495543075


In [36]:
logi_f1 = [0.6769218247670208,
  0.6762491444216291,
  0.6762491444216291,
  0.6819046077915774,
  0.6819046077915774,
  0.6819046077915774,
  0.6779962774136117,
  0.6778108173270112,
  0.6778108173270112,
  0.6776699810796668,
  0.6755764780978217,
  0.6755764780978217,
  0.6803315503654404,
  0.6803315503654404,
  0.6803315503654404, ##adult
           0.0,
  0.49220213748515634,
  0.4891782880341655,
  0.0,
  0.45826350941105043,
  0.4887124983183102,
  0.0,
  0.4529581444634132,
  0.45747680935066176,
  0.0,
  0.4572976534758575,
  0.48470460029578893,
  0.0,
  0.4735015217445985,
  0.5217045680379296, ##cov
           0.0,
  0.22280817402768624,
  0.2323592302209551,
  0.0,
  0.2241557903459706,
  0.22778025904421614,
  0.0,
  0.23085820032384918,
  0.2439266236985622,
  0.0,
  0.22586999343401182,
  0.23659454374412045,
  0.0,
  0.22783725910064243,
  0.2359963269054178, ## letterO
           0.728038428637231,
  0.728038428637231,
  0.728038428637231,
  0.728716770594027,
  0.7306200276152279,
  0.728716770594027,
  0.7284488448844885,
  0.7277902820071495,
  0.7284007656260312,
  0.7312294543063774,
  0.7313128656893039,
  0.7312294543063774,
  0.7398896462187601,
  0.7387821556496256,
  0.7398896462187601] ##letterAM
logi_f1_mean = statistics.mean(logi_f1)
print(logi_f1_mean)

0.47053162564180817


In [37]:
logi_roc = [0.8141608910114237,
  0.8152152701029196,
  0.8152152701029196,
  0.8178152089677819,
  0.8178152089677819,
  0.8178152089677819,
  0.8153525084149416,
  0.8162485410928683,
  0.8162485410928683,
  0.8145117104970799,
  0.8141157771093034,
  0.8141157771093034,
  0.8163050514649433,
  0.8163050514649433,
  0.8163050514649433, ##adult
          0.5,
  0.9253507489579847,
  0.9228363308549647,
  0.5,
  0.9253288223330063,
  0.91454295788458,
  0.5,
  0.9326755119683062,
  0.9296534793234185,
  0.5,
  0.9216495348960505,
  0.9245796265591393,
  0.5,
  0.9264297063918454,
  0.9231483726195536, ##cov
          0.5,
  0.8400094369298522,
  0.8348568732305757,
  0.5,
  0.837100276110635,
  0.8214492519730672,
  0.5,
  0.8302226578614181,
  0.8337725335939293,
  0.5,
  0.8421682038387515,
  0.8410226742878679,
  0.5,
  0.842244147591947,
  0.836739447301534, ##letterO
          0.7245518089388819,
  0.7245518089388819,
  0.7245518089388819,
  0.7246917939958031,
  0.7270749237739126,
  0.7246917939958031,
  0.7258745212418176,
  0.7259824404588612,
  0.7258084273885459,
  0.727598709811757,
  0.7276658419277613,
  0.727598709811757,
  0.733187973343531,
  0.7328897243929007,
  0.733187973343531] ##letterAM
logi_roc_mean = statistics.mean(logi_roc)
print(logi_roc_mean)

0.7625538987107143


In [38]:
svm_acc = [0.8006563572174432,
  0.7962060563870519,
  0.7985679478892148,
  0.8079409278504301,
  0.7845209089552981,
  0.7985182238575904,
  0.7853910795087266,
  0.7906121028292974,
  0.7932723385212073,
  0.7865844562677142,
  0.7736562080453483,
  0.7961563323554274,
  0.8091840286410422,
  0.800532047138382,
  0.800532047138382, ##adult
          0.9779032381269834,
  0.9305205447108741,
  0.9733373610272008,
  0.9771688784261439,
  0.9360273744296994,
  0.9771688784261439,
  0.975800851371152,
  0.9691846697638243,
  0.975800851371152,
  0.9756185635021493,
  0.9375308153302362,
  0.9756185635021493,
  0.9764084776011611,
  0.9435202738831829,
  0.9716585765574328, ##cov
          0.9927333333333334,
  0.9932,
  0.9927333333333334,
  0.9910666666666667,
  0.9911333333333333,
  0.9910666666666667,
  0.9928,
  0.9856,
  0.9882666666666666,
  0.9935333333333334,
  0.9926,
  0.9936,
  0.995,
  0.9934,
  0.995, ##letterO
          0.9633333333333334,
  0.9633333333333334,
  0.9633333333333334,
  0.9624,
  0.9624,
  0.9624,
  0.9660666666666666,
  0.9660666666666666,
  0.9660666666666666,
  0.9658,
  0.9658,
  0.9658,
  0.9648666666666667,
  0.9648666666666667,
  0.9648666666666667] ##AM
svm_acc_mean = statistics.mean(svm_acc)
print(svm_acc_mean)

0.929078871899423


In [40]:
svm_f1 = [0.646566164154104,
  0.6767617019598564,
  0.6791032953105197,
  0.6466633124456845,
  0.6673319771235559,
  0.6712906627727752,
  0.6477309826967026,
  0.6742224972922791,
  0.6681036203249112,
  0.6490883819802142,
  0.6561673842435229,
  0.6730470151932049,
  0.6445936559388746,
  0.6779334430572839,
  0.6779334430572839, ##adult
          0.6863479546574668,
  0.48165369322229273,
  0.6742459593602851,
  0.672771156286546,
  0.5018453177596627,
  0.672771156286546,
  0.6905538905538906,
  0.6554200962882435,
  0.6905538905538906,
  0.6443836726425605,
  0.4976055177806012,
  0.6443836726425605,
  0.6582673205079844,
  0.5319733567349053,
  0.6561637776701278, ##cov
          0.8991674375578169,
  0.9001956947162426,
  0.8991674375578169,
  0.8818342151675486,
  0.8717454194792671,
  0.8818342151675486,
  0.8979206049149339,
  0.7682403433476394,
  0.8540630182421227,
  0.910268270120259,
  0.9013333333333333,
  0.911275415896488,
  0.9316317228805835,
  0.9054441260744985,
  0.9316317228805835, ##letterO
          0.9631910052201846,
  0.9631910052201846,
  0.9631910052201846,
  0.9620559741657697,
  0.9620559741657697,
  0.9620559741657697,
  0.9656846221263399,
  0.9656846221263399,
  0.9656846221263399,
  0.9655033286261853,
  0.9655033286261853,
  0.9655033286261853,
  0.964403917595407,
  0.964403917595407,
  0.964403917595407] ##AM
svm_f1_mean = statistics.mean(svm_f1)
print(svm_f1_mean)

0.785395791517278


In [41]:
svm_roc = [0.7796241631839779,
  0.8188264504549373,
  0.8201244262828081,
  0.7745022999299213,
  0.8134119232564071,
  0.8086971072490617,
  0.7893221551287704,
  0.8190443057495005,
  0.8091514943555119,
  0.7889785014539118,
  0.80539153916404,
  0.8120707890189004,
  0.7725367800349026,
  0.816918659931699,
  0.816918659931699, ##adult
         0.8366170959894247,
  0.9225961008712964,
  0.8808525729807221,
  0.8268461041683141,
  0.9250145141685763,
  0.8268461041683141,
  0.8745926485462736,
  0.9026587862067245,
  0.8745926485462736,
  0.8067381621780334,
  0.9075706153569852,
  0.8067381621780334,
  0.8160135479985109,
  0.9268690178805593,
  0.8724938744415897, ##cov
          0.9402610883925763,
  0.9177665932683233,
  0.9402610883925763,
  0.9284547461342303,
  0.8888458035922434,
  0.9284547461342303,
  0.921154582210624,
  0.8175586436616691,
  0.952938171618686,
  0.9405439024775284,
  0.9530049469523667,
  0.9414415685457511,
  0.9469408089055911,
  0.9149919910998465,
  0.9469408089055911, ##O
          0.9633384721911796,
  0.9633384721911796,
  0.9633384721911796,
  0.9624042333436764,
  0.9624042333436764,
  0.9624042333436764,
  0.9660427383212338,
  0.9660427383212338,
  0.9660427383212338,
  0.9657867997778949,
  0.9657867997778949,
  0.9657867997778949,
  0.9648300659264414,
  0.9648300659264414,
  0.9648300659264414]##AM
svm_roc_mean = statistics.mean(svm_roc)
print(svm_roc_mean)

0.8899887604963133


In [42]:
rf_acc = [0.8370792103823778,
  0.8382477251255532,
  0.8368554522400676,
  0.8325294614887375,
  0.8314603948088111,
  0.835860971607578,
  0.8359852816866391,
  0.8364576599870718,
  0.8380736910148675,
  0.8315101188404356,
  0.8341454925165332,
  0.8317090149669335,
  0.8397145840584754,
  0.8382725871413654,
  0.833921734374223, ##adult
         0.9783372568627042,
  0.9755439122796052,
  0.978832038221426,
  0.9773320694707749,
  0.9757227279987223,
  0.975920640542211,
  0.9777400470823524,
  0.9743512287938446,
  0.9796688263438956,
  0.9766602084678792,
  0.9748234411783088,
  0.9752921814128872,
  0.9743547009437303,
  0.9769102032596543,
  0.9764345187253043, ##cov
         0.9884,
  0.9867333333333334,
  0.9884666666666667,
  0.9854,
  0.9851333333333333,
  0.9863333333333333,
  0.9882,
  0.9856666666666667,
  0.9881333333333333,
  0.9876,
  0.9871333333333333,
  0.9874,
  0.9891333333333333,
  0.9857333333333334,
  0.9882,           ##O
         0.9206,
  0.9186666666666666,
  0.9214666666666667,
  0.9217333333333333,
  0.9061333333333333,
  0.923,
  0.9256,
  0.9228,
  0.9202,
  0.9203333333333333,
  0.9214,
  0.92,
  0.9211333333333334,
  0.9243333333333333,
  0.9192666666666667] ##AM
rf_acc_mean = statistics.mean(rf_acc)
print(rf_acc_mean)

0.929901345252605


In [43]:
rf_f1 = [0.6403600241479611,
  0.6411472697186983,
  0.6352418010005558,
  0.6305802347263355,
  0.6243697013354019,
  0.6395107567980779,
  0.6373880063760787,
  0.6374958668577098,
  0.644971381847915,
  0.6214600904876277,
  0.6280871940681273,
  0.6220758193289041,
  0.6399731948400067,
  0.6384303262742482,
  0.6251823588822804, ##adult
        0.6093544549495962,
  0.5138558166821962,
  0.6286132009381378,
  0.5841587311697825,
  0.5254835425856803,
  0.5442597095353882,
  0.5970712085978255,
  0.47084527220630373,
  0.6464817218583029,
  0.5578795053933175,
  0.5017179769103903,
  0.5239178430454272,
  0.48346038184488427,
  0.5583449558344955,
  0.5357729138166895, ## cov
        0.8238866396761133,
  0.78625134264232,
  0.8243654822335026,
  0.7735263702171664,
  0.7684319833852544,
  0.790602655771195,
  0.8217522658610272,
  0.7724867724867724,
  0.8244575936883629,
  0.811740890688259,
  0.7991675338189386,
  0.8029197080291971,
  0.8371628371628371,
  0.773784355179704,
  0.8238805970149254, ##O
        0.9189520244981286,
  0.9165298303229337,
  0.9199728260869565,
  0.9198415949747372,
  0.903177004538578,
  0.9214979949704343,
  0.923853711790393,
  0.9206631953959988,
  0.9179968486675344,
  0.917852478174194,
  0.9188966086537801,
  0.9177856947108797,
  0.9185430007574191,
  0.9223188009034289,
  0.9168097822353507] ##AM
rf_f1_mean = statistics.mean(rf_f1)
print(rf_f1_mean)

0.7266099947765778


In [44]:
rf_roc = [0.753100293779156,
  0.753132497654158,
  0.7488948445206453,
  0.7465690814567973,
  0.7419998053703764,
  0.7523746352793929,
  0.7508884595424785,
  0.7507305321236742,
  0.756252847890528,
  0.739411389727349,
  0.7435063218990879,
  0.7398113097433457,
  0.7512658458690523,
  0.7507472468454419,
  0.7420475382050769, ##adult
         0.7375286635108119,
  0.6819249928743367,
  0.7516851959624034,
  0.7238863584042314,
  0.6892097295527458,
  0.7020861995945574,
  0.73204338041861,
  0.6608696479954992,
  0.7616922196920412,
  0.7069101031689757,
  0.6780149873092356,
  0.6907980245423058,
  0.6687183834692195,
  0.7054224458001981,
  0.6915829555580741, ##cov
         0.8689273356401384,
  0.8322082415854042,
  0.8680874488832967,
  0.8214427464049574,
  0.818000405613219,
  0.8326650304466556,
  0.8615826514441197,
  0.8235675405273429,
  0.8700826805409576,
  0.8589255279622701,
  0.8440113932674859,
  0.8450129158752085,
  0.8665189983346197,
  0.8201784958512356,
  0.8618288426972389, ##O
         0.9205206085646004,
  0.9185605404696967,
  0.9213957421413627,
  0.921590197829748,
  0.9059202634553365,
  0.9229010632217282,
  0.9254707575813472,
  0.9226358779837242,
  0.9200362605013445,
  0.9201681801734967,
  0.921230275674052,
  0.9198574807933878,
  0.9209148236733536,
  0.9241706532313412,
  0.9190694494061238] ##AM
rf_roc_mean = statistics.mean(rf_roc)
print(rf_roc_mean)

0.8050098061255767


In [45]:
logi_adult = [0.8026204564666103,
  0.8000845308537616,
  0.8000845308537616,
  0.8045099696683408,
  0.8045099696683408,
  0.8045099696683408,
  0.8021480781661777,
  0.8006563572174432,
  0.8006563572174432,
  0.8009298393913779,
  0.7978220874148476,
  0.7978220874148476,
  0.805355278205957,
  0.805355278205957,
  0.805355278205957, ##acc
             0.6769218247670208,
  0.6762491444216291,
  0.6762491444216291,
  0.6819046077915774,
  0.6819046077915774,
  0.6819046077915774,
  0.6779962774136117,
  0.6778108173270112,
  0.6778108173270112,
  0.6776699810796668,
  0.6755764780978217,
  0.6755764780978217,
  0.6803315503654404,
  0.6803315503654404,
  0.6803315503654404, ##f1
             0.8141608910114237,
  0.8152152701029196,
  0.8152152701029196,
  0.8178152089677819,
  0.8178152089677819,
  0.8178152089677819,
  0.8153525084149416,
  0.8162485410928683,
  0.8162485410928683,
  0.8145117104970799,
  0.8141157771093034,
  0.8141157771093034,
  0.8163050514649433,
  0.8163050514649433,
  0.8163050514649433] ##roc
logi_adult_mean = statistics.mean(logi_adult)
print(logi_adult_mean)

0.7655229905305609


In [46]:
logi_cov = [0.9646847635118713,
  0.9331854197481997,
  0.932720151663507,
  0.9647125407109574,
  0.9225502246480977,
  0.9340222078706694,
  0.9647368457601577,
  0.919336749928821,
  0.9215137879071964,
  0.9646569863127852,
  0.9227880669152726,
  0.9310431032686819,
  0.9646864995868142,
  0.9273209585911405,
  0.9415046908744956, ##acc
 0.0,
  0.49220213748515634,
  0.4891782880341655,
  0.0,
  0.45826350941105043,
  0.4887124983183102,
  0.0,
  0.4529581444634132,
  0.45747680935066176,
  0.0,
  0.4572976534758575,
  0.48470460029578893,
  0.0,
  0.4735015217445985,
  0.5217045680379296, ##f1
 0.5,
  0.9253507489579847,
  0.9228363308549647,
  0.5,
  0.9253288223330063,
  0.91454295788458,
  0.5,
  0.9326755119683062,
  0.9296534793234185,
  0.5,
  0.9216495348960505,
  0.9245796265591393,
  0.5,
  0.9264297063918454,
  0.9231483726195536] ##roc
logi_cov_mean = statistics.mean(logi_cov)
print(logi_cov_mean)

0.6807035071045433


In [48]:
logi_letterO = [0.9633333333333334,
  0.7642,
  0.7846,
  0.9612666666666667,
  0.7503333333333333,
  0.7694666666666666,
  0.9624666666666667,
  0.7783333333333333,
  0.7966666666666666,
  0.9628666666666666,
  0.7642,
  0.7836,
  0.9619333333333333,
  0.7596,
  0.7781333333333333, ##acc
 0.0,
  0.22280817402768624,
  0.2323592302209551,
  0.0,
  0.2241557903459706,
  0.22778025904421614,
  0.0,
  0.23085820032384918,
  0.2439266236985622,
  0.0,
  0.22586999343401182,
  0.23659454374412045,
  0.0,
  0.22783725910064243,
  0.2359963269054178, ##f1
 0.5,
  0.8400094369298522,
  0.8348568732305757,
  0.5,
  0.837100276110635,
  0.8214492519730672,
  0.5,
  0.8302226578614181,
  0.8337725335939293,
  0.5,
  0.8421682038387515,
  0.8410226742878679,
  0.5,
  0.842244147591947,
  0.836739447301534] ##roc
logi_letterO_mean = statistics.mean(logi_letterO)
print(logi_letterO_mean)

0.5713060423014447


In [49]:
logi_letterAM = [0.7244666666666667,
  0.7244666666666667,
  0.7244666666666667,
  0.7244666666666667,
  0.7268666666666667,
  0.7244666666666667,
  0.7257333333333333,
  0.7258666666666667,
  0.7256666666666667,
  0.7274666666666667,
  0.7275333333333334,
  0.7274666666666667,
  0.7328666666666667,
  0.7326,
  0.7328666666666667, ##acc
 0.728038428637231,
  0.728038428637231,
  0.728038428637231,
  0.728716770594027,
  0.7306200276152279,
  0.728716770594027,
  0.7284488448844885,
  0.7277902820071495,
  0.7284007656260312,
  0.7312294543063774,
  0.7313128656893039,
  0.7312294543063774,
  0.7398896462187601,
  0.7387821556496256,
  0.7398896462187601, ##f1
 0.7245518089388819,
  0.7245518089388819,
  0.7245518089388819,
  0.7246917939958031,
  0.7270749237739126,
  0.7246917939958031,
  0.7258745212418176,
  0.7259824404588612,
  0.7258084273885459,
  0.727598709811757,
  0.7276658419277613,
  0.727598709811757,
  0.733187973343531,
  0.7328897243929007,
  0.733187973343531] ##roc
logi_letterAM_mean = statistics.mean(logi_letterAM)
print(logi_letterAM_mean)

0.7285848199242476


In [50]:
svm_adult = [0.8006563572174432,
  0.7962060563870519,
  0.7985679478892148,
  0.8079409278504301,
  0.7845209089552981,
  0.7985182238575904,
  0.7853910795087266,
  0.7906121028292974,
  0.7932723385212073,
  0.7865844562677142,
  0.7736562080453483,
  0.7961563323554274,
  0.8091840286410422,
  0.800532047138382,
  0.800532047138382, ##acc
 0.646566164154104,
  0.6767617019598564,
  0.6791032953105197,
  0.6466633124456845,
  0.6673319771235559,
  0.6712906627727752,
  0.6477309826967026,
  0.6742224972922791,
  0.6681036203249112,
  0.6490883819802142,
  0.6561673842435229,
  0.6730470151932049,
  0.6445936559388746,
  0.6779334430572839,
  0.6779334430572839, ##f1
 0.7796241631839779,
  0.8188264504549373,
  0.8201244262828081,
  0.7745022999299213,
  0.8134119232564071,
  0.8086971072490617,
  0.7893221551287704,
  0.8190443057495005,
  0.8091514943555119,
  0.7889785014539118,
  0.80539153916404,
  0.8120707890189004,
  0.7725367800349026,
  0.816918659931699,
  0.816918659931699] ##roc
svm_adult_mean = statistics.mean(svm_adult)
print(svm_adult_mean)

0.7538752856728751


In [51]:
svm_cov = [0.9779032381269834,
  0.9305205447108741,
  0.9733373610272008,
  0.9771688784261439,
  0.9360273744296994,
  0.9771688784261439,
  0.975800851371152,
  0.9691846697638243,
  0.975800851371152,
  0.9756185635021493,
  0.9375308153302362,
  0.9756185635021493,
  0.9764084776011611,
  0.9435202738831829,
  0.9716585765574328, ##acc
 0.6863479546574668,
  0.48165369322229273,
  0.6742459593602851,
  0.672771156286546,
  0.5018453177596627,
  0.672771156286546,
  0.6905538905538906,
  0.6554200962882435,
  0.6905538905538906,
  0.6443836726425605,
  0.4976055177806012,
  0.6443836726425605,
  0.6582673205079844,
  0.5319733567349053,
  0.6561637776701278, ##f1
 0.8366170959894247,
  0.9225961008712964,
  0.8808525729807221,
  0.8268461041683141,
  0.9250145141685763,
  0.8268461041683141,
  0.8745926485462736,
  0.9026587862067245,
  0.8745926485462736,
  0.8067381621780334,
  0.9075706153569852,
  0.8067381621780334,
  0.8160135479985109,
  0.9268690178805593,
  0.8724938744415897] ##roc
svm_cov_mean = statistics.mean(svm_cov)
print(svm_cov_mean)

0.8186499623701484


In [52]:
svm_letterO = [0.9927333333333334,
  0.9932,
  0.9927333333333334,
  0.9910666666666667,
  0.9911333333333333,
  0.9910666666666667,
  0.9928,
  0.9856,
  0.9882666666666666,
  0.9935333333333334,
  0.9926,
  0.9936,
  0.995,
  0.9934,
  0.995,            ##acc
 0.8991674375578169,
  0.9001956947162426,
  0.8991674375578169,
  0.8818342151675486,
  0.8717454194792671,
  0.8818342151675486,
  0.8979206049149339,
  0.7682403433476394,
  0.8540630182421227,
  0.910268270120259,
  0.9013333333333333,
  0.911275415896488,
  0.9316317228805835,
  0.9054441260744985,
  0.9316317228805835, ##f1
 0.9402610883925763,
  0.9177665932683233,
  0.9402610883925763,
  0.9284547461342303,
  0.8888458035922434,
  0.9284547461342303,
  0.921154582210624,
  0.8175586436616691,
  0.952938171618686,
  0.9405439024775284,
  0.9530049469523667,
  0.9414415685457511,
  0.9469408089055911,
  0.9149919910998465,
  0.9469408089055911] ##roc
svm_letterO_mean = statistics.mean(svm_letterO)
print(svm_letterO_mean)

0.9357121289102633


In [53]:
svm_letterAM = [0.9633333333333334,
  0.9633333333333334,
  0.9633333333333334,
  0.9624,
  0.9624,
  0.9624,
  0.9660666666666666,
  0.9660666666666666,
  0.9660666666666666,
  0.9658,
  0.9658,
  0.9658,
  0.9648666666666667,
  0.9648666666666667,
  0.9648666666666667, #acc
 0.9631910052201846,
  0.9631910052201846,
  0.9631910052201846,
  0.9620559741657697,
  0.9620559741657697,
  0.9620559741657697,
  0.9656846221263399,
  0.9656846221263399,
  0.9656846221263399,
  0.9655033286261853,
  0.9655033286261853,
  0.9655033286261853,
  0.964403917595407,
  0.964403917595407,
  0.964403917595407, #f1
 0.9633384721911796,
  0.9633384721911796,
  0.9633384721911796,
  0.9624042333436764,
  0.9624042333436764,
  0.9624042333436764,
  0.9660427383212338,
  0.9660427383212338,
  0.9660427383212338,
  0.9657867997778949,
  0.9657867997778949,
  0.9657867997778949,
  0.9648300659264414,
  0.9648300659264414,
  0.9648300659264414] #roc
svm_letterAM_mean = statistics.mean(svm_letterAM)
print(svm_letterAM_mean)

0.9643805215973986


In [54]:
rf_adult = [0.8370792103823778,
  0.8382477251255532,
  0.8368554522400676,
  0.8325294614887375,
  0.8314603948088111,
  0.835860971607578,
  0.8359852816866391,
  0.8364576599870718,
  0.8380736910148675,
  0.8315101188404356,
  0.8341454925165332,
  0.8317090149669335,
  0.8397145840584754,
  0.8382725871413654,
  0.833921734374223, #acc
 0.6403600241479611,
  0.6411472697186983,
  0.6352418010005558,
  0.6305802347263355,
  0.6243697013354019,
  0.6395107567980779,
  0.6373880063760787,
  0.6374958668577098,
  0.644971381847915,
  0.6214600904876277,
  0.6280871940681273,
  0.6220758193289041,
  0.6399731948400067,
  0.6384303262742482,
  0.6251823588822804, #f1
 0.753100293779156,
  0.753132497654158,
  0.7488948445206453,
  0.7465690814567973,
  0.7419998053703764,
  0.7523746352793929,
  0.7508884595424785,
  0.7507305321236742,
  0.756252847890528,
  0.739411389727349,
  0.7435063218990879,
  0.7398113097433457,
  0.7512658458690523,
  0.7507472468454419,
  0.7420475382050769] #roc
rf_adult_mean = statistics.mean(rf_adult)
print(rf_adult_mean)

0.7390851123741369


In [55]:
rf_cov = [0.9783372568627042,
  0.9755439122796052,
  0.978832038221426,
  0.9773320694707749,
  0.9757227279987223,
  0.975920640542211,
  0.9777400470823524,
  0.9743512287938446,
  0.9796688263438956,
  0.9766602084678792,
  0.9748234411783088,
  0.9752921814128872,
  0.9743547009437303,
  0.9769102032596543,
  0.9764345187253043, #acc
 0.6093544549495962,
  0.5138558166821962,
  0.6286132009381378,
  0.5841587311697825,
  0.5254835425856803,
  0.5442597095353882,
  0.5970712085978255,
  0.47084527220630373,
  0.6464817218583029,
  0.5578795053933175,
  0.5017179769103903,
  0.5239178430454272,
  0.48346038184488427,
  0.5583449558344955,
  0.5357729138166895, #f1
 0.7375286635108119,
  0.6819249928743367,
  0.7516851959624034,
  0.7238863584042314,
  0.6892097295527458,
  0.7020861995945574,
  0.73204338041861,
  0.6608696479954992,
  0.7616922196920412,
  0.7069101031689757,
  0.6780149873092356,
  0.6907980245423058,
  0.6687183834692195,
  0.7054224458001981,
  0.6915829555580741] #roc
rf_cov_mean =statistics.mean(rf_cov)
print(rf_cov_mean)

0.7447003227734437


In [56]:
rf_letterO = [0.9884,
  0.9867333333333334,
  0.9884666666666667,
  0.9854,
  0.9851333333333333,
  0.9863333333333333,
  0.9882,
  0.9856666666666667,
  0.9881333333333333,
  0.9876,
  0.9871333333333333,
  0.9874,
  0.9891333333333333,
  0.9857333333333334,
  0.9882,            #acc
 0.8238866396761133,
  0.78625134264232,
  0.8243654822335026,
  0.7735263702171664,
  0.7684319833852544,
  0.790602655771195,
  0.8217522658610272,
  0.7724867724867724,
  0.8244575936883629,
  0.811740890688259,
  0.7991675338189386,
  0.8029197080291971,
  0.8371628371628371,
  0.773784355179704,
  0.8238805970149254, #f1
0.8689273356401384,
  0.8322082415854042,
  0.8680874488832967,
  0.8214427464049574,
  0.818000405613219,
  0.8326650304466556,
  0.8615826514441197,
  0.8235675405273429,
  0.8700826805409576,
  0.8589255279622701,
  0.8440113932674859,
  0.8450129158752085,
  0.8665189983346197,
  0.8201784958512356,
  0.8618288426972389] #roc
rf_letterO_mean =statistics.mean(rf_letterO)
print(rf_letterO_mean)

0.8785583099910309


In [58]:
rf_letterAM = [0.9206,
  0.9186666666666666,
  0.9214666666666667,
  0.9217333333333333,
  0.9061333333333333,
  0.923,
  0.9256,
  0.9228,
  0.9202,
  0.9203333333333333,
  0.9214,
  0.92,
  0.9211333333333334,
  0.9243333333333333,
  0.9192666666666667, #acc
 0.9189520244981286,
  0.9165298303229337,
  0.9199728260869565,
  0.9198415949747372,
  0.903177004538578,
  0.9214979949704343,
  0.923853711790393,
  0.9206631953959988,
  0.9179968486675344,
  0.917852478174194,
  0.9188966086537801,
  0.9177856947108797,
  0.9185430007574191,
  0.9223188009034289,
  0.9168097822353507, #f1
 0.9205206085646004,
  0.9185605404696967,
  0.9213957421413627,
  0.921590197829748,
  0.9059202634553365,
  0.9229010632217282,
  0.9254707575813472,
  0.9226358779837242,
  0.9200362605013445,
  0.9201681801734967,
  0.921230275674052,
  0.9198574807933878,
  0.9209148236733536,
  0.9241706532313412,
  0.9190694494061238] #roc
rf_letterAM_mean=statistics.mean(rf_letterAM)
print(rf_letterAM_mean)

0.9196844497344012


In [86]:
t, p = ttest_ind(logi_acc, rf_acc)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -7.36624  p = 2.59607e-11


In [85]:
t, p = ttest_ind(svm_acc, rf_acc)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -0.0636439  p = 0.949361


In [84]:
t, p = ttest_ind(logi_f1, svm_f1)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -7.76095  p = 3.36259e-12


In [83]:
t, p = ttest_ind(rf_f1, svm_f1)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -2.15508  p = 0.0331874


In [87]:
t, p = ttest_ind(logi_roc, svm_roc)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -6.55232  p = 1.55438e-09


In [88]:
t, p = ttest_ind(rf_roc, svm_roc)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -5.99105  p = 2.31829e-08


In [90]:
t, p = ttest_ind(svm_adult, logi_adult)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -0.860426  p = 0.391892


In [92]:
t, p = ttest_ind(rf_adult, logi_adult)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -1.69799  p = 0.0930434


In [93]:
t, p = ttest_ind(logi_cov, svm_cov)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -2.60543  p = 0.010773


In [94]:
t, p = ttest_ind(rf_cov, svm_cov)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -2.09077  p = 0.0394324


In [95]:
t, p = ttest_ind(logi_letterO, svm_letterO)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -7.3934  p = 7.88752e-11


In [96]:
t, p = ttest_ind(rf_letterO, svm_letterO)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -3.95952  p = 0.000152014


In [97]:
t, p = ttest_ind(logi_letterAM, svm_letterAM)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -368.354  p = 4.38636e-142


In [98]:
t, p = ttest_ind(rf_letterAM, svm_letterAM)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -63.7713  p = 1.87982e-75


In [101]:
##training set performance
logi_adult_tr = [0.8096,
  0.804,
  0.804,
  0.8074,
  0.8074,
  0.8074,
  0.8162,
  0.814,
  0.814,
  0.8062,
  0.8018,
  0.8018,
  0.8076,
  0.8076,
  0.8076, #acc
 0.6934964584674822,
  0.688096753660089,
  0.688096753660089,
  0.6847790507364975,
  0.6847790507364975,
  0.6847790507364975,
  0.6987872828580793,
  0.6962769431743958,
  0.6962769431743958,
  0.6800924397490921,
  0.6751884627990824,
  0.6751884627990824,
  0.6890756302521008,
  0.6890756302521008,
  0.6890756302521008, #f1
 0.8220463127476628,
  0.8193291823142196,
  0.8193291823142196,
  0.8230977159075551,
  0.8230977159075551,
  0.8230977159075551,
  0.8279333642479396,
  0.826465914941643,
  0.826465914941643,
  0.8225243926213249,
  0.8196250779138922,
  0.8196250779138922,
  0.8189757076216079,
  0.8189757076216079,
  0.8189757076216079] #roc
logi_adult_tr_mean = statistics.mean(logi_adult_tr)
print(logi_adult_tr_mean)

0.7724273163078113


In [102]:
svm_adult_tr =  [0.802,
  0.8054,
  0.813,
  0.8046,
  0.7946,
  0.8624,
  0.7916,
  0.8072,
  0.8422,
  0.7938,
  0.7842,
  0.8372,
  0.806,
  0.806,
  0.806,
 0.6564885496183207,
  0.695270905104917,
  0.706436420722135,
  0.6374768089053804,
  0.6777533730781299,
  0.7733860342555995,
  0.6610279765777489,
  0.69875,
  0.7462206497266002,
  0.6503899626992201,
  0.6656337155252556,
  0.7315303430079155,
  0.643906020558003,
  0.6926489226869456,
  0.6926489226869456,
 0.7829819187818952,
  0.8277876696966953,
  0.8367724718692182,
  0.7696850925800783,
  0.8239951344776321,
  0.8948128125422354,
  0.7982267859152145,
  0.8357599026189899,
  0.870275861716238,
  0.7942743233402484,
  0.8204713123890301,
  0.8652931092210709,
  0.7685679012972463,
  0.824952551634737,
  0.824952551634737]
svm_adult_tr_mean = statistics.mean(svm_adult_tr)
print(svm_adult_tr_mean)

0.7738795112192974


In [103]:
rf_adult_tr = [0.9818,
  0.9822,
  0.98,
  0.982,
  0.9814,
  0.9806,
  0.9818,
  0.9842,
  0.9828,
  0.9798,
  0.9802,
  0.984,
  0.9806,
  0.9796,
  0.9798,
 0.9637305699481865,
  0.9643572286744093,
  0.9600000000000001,
  0.9627174813587407,
  0.961617829137433,
  0.9599007854485325,
  0.9631130928252939,
  0.9679773003648157,
  0.9650122050447518,
  0.9573659772055721,
  0.9584208315833683,
  0.9667221297836939,
  0.9610285255122538,
  0.9591999999999999,
  0.9592577652279145,
 0.9717213141560216,
  0.9706929484367639,
  0.9681807948798611,
  0.9707095553453169,
  0.9711393431544803,
  0.9695066900932559,
  0.9708389735441436,
  0.9740355665726727,
  0.9709740828761495,
  0.9663274297988182,
  0.9685706505252687,
  0.977013544122648,
  0.9687384446413174,
  0.9688520414854829,
  0.9663755456667774]
rf_adult_tr_mean = statistics.mean(rf_adult_tr)
print(rf_adult_tr_mean)

0.9712199699425321


In [104]:
t, p = ttest_ind(logi_adult, logi_adult_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -0.529647  p = 0.597691


In [105]:
t, p = ttest_ind(svm_adult, svm_adult_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -1.40444  p = 0.163708


In [106]:
t, p = ttest_ind(rf_adult, rf_adult_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -18.5104  p = 4.29884e-32


In [107]:
logi_cov_tr = [0.9664,
  0.9344,
  0.9338,
  0.9632,
  0.9244,
  0.9364,
  0.9604,
  0.9182,
  0.922,
  0.9696,
  0.929,
  0.9368,
  0.9662,
  0.9314,
  0.9452,
 0.0,
  0.49693251533742333,
  0.4946564885496183,
  0.0,
  0.4821917808219179,
  0.5137614678899083,
  0.0,
  0.4803049555273189,
  0.49350649350649345,
  0.0,
  0.4496124031007752,
  0.46801346801346805,
  0.0,
  0.4763358778625954,
  0.5243055555555556,
 0.5,
  0.9488233207190161,
  0.9485128902554399,
  0.5,
  0.9398472483027589,
  0.9251679185324282,
  0.5,
  0.9356234144863882,
  0.940022886087026,
  0.5,
  0.9410825951016155,
  0.9259868421052632,
  0.5,
  0.9273840421635909,
  0.9202500125545203]
logi_cov_tr_mean = statistics.mean(logi_cov_tr)
print(logi_cov_tr_mean)

0.6859938261438472


In [108]:
t, p = ttest_ind(logi_cov, logi_cov_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -0.0782675  p = 0.937793


In [109]:
svm_cov_tr = [0.993,
  0.9322,
  0.9872,
  0.9994,
  0.9444,
  0.9994,
  0.9922,
  0.9816,
  0.9922,
  0.9984,
  0.9422,
  0.9984,
  0.9982,
  0.9522,
  0.9882,
 0.9056603773584906,
  0.4932735426008969,
  0.8400000000000001,
  0.9919137466307278,
  0.5683229813664596,
  0.9919137466307278,
  0.9103448275862069,
  0.8114754098360656,
  0.9103448275862069,
  0.9743589743589743,
  0.5109983079526227,
  0.9743589743589743,
  0.9740634005763689,
  0.58578856152513,
  0.8513853904282116,
 0.9963783112582781,
  0.9563032166508988,
  0.9933774834437087,
  0.9996885382059801,
  0.9685243030478116,
  0.9996885382059801,
  0.9959391920033319,
  0.990420658059142,
  0.9959391920033319,
  0.9991749174917492,
  0.9670075560187598,
  0.9991749174917492,
  0.9990685158352308,
  0.9752639205133512,
  0.9938936038087353]
svm_cov_tr_mean = statistics.mean(svm_cov_tr)
print(svm_cov_tr_mean)

0.9294054651740912


In [110]:
t, p = ttest_ind(svm_cov, svm_cov_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -3.67435  p = 0.000409151


In [111]:
rf_cov_tr = [0.998,
  0.9978,
  0.9976,
  0.9976,
  0.9974,
  0.9974,
  0.998,
  0.9974,
  0.9974,
  0.9972,
  0.9982,
  0.9974,
  0.9978,
  0.9978,
  0.9966,
 0.9693251533742331,
  0.9661538461538461,
  0.9634146341463415,
  0.9662921348314606,
  0.9633802816901409,
  0.9633802816901409,
  0.9740932642487047,
  0.9660574412532638,
  0.9662337662337662,
  0.952054794520548,
  0.9694915254237287,
  0.9553264604810997,
  0.9663608562691132,
  0.9663608562691132,
  0.9470404984423676,
 0.9702380952380952,
  0.9672619047619048,
  0.9700311415957111,
  0.9673913043478262,
  0.9646739130434783,
  0.9646739130434783,
  0.9747474747474747,
  0.9671717171717171,
  0.9695928464150039,
  0.9571337067917318,
  0.9703947368421053,
  0.9572368421052632,
  0.9674556213017751,
  0.9674556213017751,
  0.9497041420118344]
rf_cov_tr_mean = statistics.mean(rf_cov_tr)
print(rf_cov_tr_mean)

0.9756384172388232


In [112]:
t, p = ttest_ind(rf_cov, rf_cov_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -8.53698  p = 3.68057e-13


In [115]:
logi_letterO_tr = [0.9594,
  0.76,
  0.781,
  0.9656,
  0.7518,
  0.774,
  0.962,
  0.7826,
  0.7984,
  0.9608,
  0.772,
  0.7894,
  0.9636,
  0.753,
  0.7714,
 0.0,
  0.2433795712484237,
  0.2545949625595643,
  0.0,
  0.20499679692504805,
  0.21636615811373094,
  0.0,
  0.24356297842727906,
  0.2555391432791728,
  0.0,
  0.2440318302387268,
  0.2579281183932347,
  0.0,
  0.21587301587301586,
  0.2271805273833671,
 0.5,
  0.851333602384906,
  0.8481250083436795,
  0.5,
  0.8378379029305794,
  0.8381196894086819,
  0.5,
  0.8490918043549622,
  0.8522486048801838,
  0.5,
  0.8519855902394264,
  0.8585936039694813,
  0.5,
  0.8401130832637682,
  0.8443736628668136]
logi_letterO_tr_mean = statistics.mean(logi_letterO_tr)
print(logi_letterO_tr_mean)

0.5751172367796454


In [116]:
t, p = ttest_ind(logi_letterO, logi_letterO_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -0.0552874  p = 0.956035


In [117]:
svm_letterO_tr =[1.0,
  1.0,
  1.0,
  0.9994,
  1.0,
  0.9994,
  1.0,
  1.0,
  0.9966,
  1.0,
  0.9976,
  1.0,
  1.0,
  1.0,
  1.0,
 1.0,
  1.0,
  1.0,
  0.9913544668587896,
  1.0,
  0.9913544668587896,
  1.0,
  1.0,
  0.9571788413098236,
  1.0,
  0.9702970297029703,
  1.0,
  1.0,
  1.0,
  1.0,
 1.0,
  1.0,
  1.0,
  0.9996893123446562,
  1.0,
  0.9996893123446562,
  1.0,
  1.0,
  0.9982328482328483,
  1.0,
  0.998751040799334,
  1.0,
  1.0,
  1.0,
  1.0]
svm_letterO_tr_mean = statistics.mean(svm_letterO_tr)
print(svm_letterO_tr_mean)

0.9977677181878193


In [118]:
t, p = ttest_ind(svm_letterO, svm_letterO_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -7.90597  p = 7.20806e-12


In [119]:
rf_letterO_tr =[0.9982,
  0.9986,
  0.9988,
  0.9988,
  0.999,
  0.9988,
  0.9984,
  0.9986,
  0.9988,
  0.9992,
  0.9988,
  0.9988,
  0.9982,
  0.998,
  0.9986,
 0.9773299748110832,
  0.9824561403508771,
  0.985,
  0.9822485207100593,
  0.9852507374631269,
  0.9822485207100593,
  0.978494623655914,
  0.9812332439678284,
  0.983957219251337,
  0.9897435897435899,
  0.9844559585492229,
  0.9844559585492229,
  0.9747899159663866,
  0.9717514124293786,
  0.9803921568627451,
 0.9778325123152709,
  0.9827586206896552,
  0.9852216748768473,
  0.9825581395348837,
  0.9854651162790697,
  0.9825581395348837,
  0.9789473684210526,
  0.9815789473684211,
  0.9842105263157894,
  0.9922428588421214,
  0.9846938775510203,
  0.9846938775510203,
  0.9779182005209403,
  0.9725274725274725,
  0.9807692307692308]
rf_letterO_tr_mean = statistics.mean(rf_letterO_tr)
print(rf_letterO_tr_mean)

0.9874974341359669


In [120]:
t, p = ttest_ind(rf_letterO, rf_letterO_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -8.90103  p = 6.56269e-14


In [122]:
logi_letterAM_tr =[0.7358,
  0.736,
  0.7358,
  0.7372,
  0.7376,
  0.7372,
  0.7214,
  0.7212,
  0.7216,
  0.7322,
  0.7322,
  0.7322,
  0.729,
  0.7292,
  0.729,
 0.7388812018185412,
  0.7390272835112693,
  0.7388812018185412,
  0.7414403778040143,
  0.7409162717219591,
  0.7414403778040143,
  0.7264873355586098,
  0.7253743104806935,
  0.7266300078554596,
  0.7359495168605799,
  0.7359495168605799,
  0.7359495168605799,
  0.7344699196551048,
  0.7341970946211229,
  0.7344699196551048,
 0.7359608328068926,
  0.7361590881597871,
  0.7359608328068926,
  0.7371312764368292,
  0.7375514496134599,
  0.7371312764368292,
  0.7213656854627417,
  0.7211724846899388,
  0.721566086264345,
  0.732252938270088,
  0.732252938270088,
  0.732252938270088,
  0.7289234719584171,
  0.7291313959838261,
  0.7289234719584171]
logi_letterAM_tr_mean = statistics.mean(logi_letterAM_tr)
print(logi_letterAM_tr_mean)

0.7325644448949958


In [123]:
t, p = ttest_ind(logi_letterAM, logi_letterAM_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -3.71802  p = 0.00035263


In [124]:
svm_letterAM_tr=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
svm_letterAM_tr_mean = statistics.mean(svm_letterAM_tr)
print(svm_letterAM_tr_mean)

1.0


In [125]:
t, p = ttest_ind(svm_letterAM, svm_letterAM_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -167.551  p = 5.03061e-112


In [126]:
rf_letterAM_tr=[0.998,
  0.9968,
  0.9964,
  0.9978,
  0.9966,
  0.996,
  0.9982,
  0.998,
  0.9978,
  0.9968,
  0.9974,
  0.999,
  0.9962,
  0.9984,
  0.9962,
 0.9979822437449556,
  0.9967611336032389,
  0.9963577498988264,
  0.9978083283522614,
  0.9966142202748456,
  0.9960111687275628,
  0.9982017982017982,
  0.998003992015968,
  0.9978021978021978,
  0.9967832730197024,
  0.9973858837723707,
  0.9989965884005618,
  0.9962068277101218,
  0.9984063745019921,
  0.9962083416483736,
0.9979998451080052,
 0.9967715899919289,
  0.9963750792861399,
  0.997810011321955,
  0.9966077336185264,
  0.9960155550478064,
  0.9982019928079713,
  0.9979999919999679,
  0.9978019912079649,
  0.9967935671661278,
  0.9973929333036371,
  0.9989993497533415,
  0.9962113666520583,
  0.9984017290574745,
  0.9962096066179842]
rf_letterAM_tr_mean = statistics.mean(rf_letterAM_tr)
print(rf_letterAM_tr_mean)

0.9973049436581259


In [127]:
t, p = ttest_ind(rf_letterAM, rf_letterAM_tr)
print("ttest_ind:   t = %g  p = %g" % (t, p))

ttest_ind:   t = -113.863  p = 2.48964e-97
